In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn

Mounted at /content/drive


In [ ]:
news_press = pd.read_csv('/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/NEWS_+_PRESS_ESG_definitivo.csv')

In [ ]:
summary = pd.read_csv('/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/DF_SUMMARY.csv')

In [ ]:
path = "/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/EMBEDDINGS.pkl"
embeddings = pd.read_pickle(path)

In [ ]:
df_preds = pd.read_csv('/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/DAILY_ESG_PREDICTIONS.csv')
df_preds

,ticker,category,target_date,predicted_score
0,AAPL,Environmental,2020-07-01,72.048330
1,AAPL,Environmental,2020-07-02,72.323980
2,AAPL,Environmental,2020-07-03,72.354250
3,AAPL,Environmental,2020-07-05,72.138850
4,AAPL,Environmental,2020-07-06,72.300810
...,...,...,...,...
63396,XOM,Social,2024-11-01,73.365210
63397,XOM,Social,2024-11-08,73.563446
63398,XOM,Social,2024-11-15,73.662570
63399,XOM,Social,2024-11-27,74.006615


In [ ]:
df_embeddings = embeddings.copy()
df_embeddings["embedding_combined"] = df_embeddings["embedding_combined"].apply(lambda x: np.array(x))
df_embeddings["date"] = pd.to_datetime(df_embeddings["date"])
df_embeddings = df_embeddings.sort_values(["ticker", "category", "date"]).reset_index(drop=True)

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def generate_windows_with_padding(df, window_size=7, max_padding_fraction=0.2):
    X, y, tickers, categories, target_dates = [], [], [], [], []

    min_required_days = int(window_size * (1 - max_padding_fraction))

    grouped = df.groupby(["ticker", "category"])

    for (ticker, category), group in tqdm(grouped, desc="Generando ventanas con padding"):
        group = group.sort_values("date").reset_index(drop=True)

        if len(group) < min_required_days:
            continue
        for i in range(min_required_days, len(group) + 1):
            window = group.iloc[max(0, i - window_size):i]

            embeddings = window["embedding_combined"].values
            embeddings = np.stack(embeddings)

            if embeddings.shape[0] < window_size:
                missing = window_size - embeddings.shape[0]
                padding = np.zeros((missing, embeddings.shape[1]))
                embeddings = np.vstack([padding, embeddings])

            X.append(embeddings)

            y.append(None)
            tickers.append(ticker)
            categories.append(category)
            target_dates.append(window.iloc[-1]["date"])

    return np.array(X), y, tickers, categories, target_dates

In [ ]:
X_7d, _, tickers_7d, categories_7d, target_dates_7d = generate_windows_with_padding(
    df_embeddings,
    window_size=7,
    max_padding_fraction=0.2
)

Generando ventanas con padding:   0%|          | 0/90 [00:00<?, ?it/s]

In [ ]:
df_preds = df_preds.rename(columns={"target_date": "date"})

# Generating Short Rolling Windows with Padding

We define a function `generate_windows_with_padding` to construct shorter temporal windows:
- **Window size**: 7 days.
- **Maximum padding allowed**: 20% (i.e., at least 6 out of 7 days must be real data).

For each company (`ticker`) and ESG category:
- If sufficient daily embeddings are available, we create overlapping 7-day windows.
- If the window lacks days, we apply **zero-padding at the beginning** to preserve window length.

Each window is associated with:
- A sequence of embeddings.
- Metadata (`ticker`, `category`, `target_date`).

Why allow padding?
- Real-world ESG news flow may be irregular.
- Allowing up to 20% missing days balances realism and model robustness.

This structure enables daily dynamic modeling of ESG evolution at a much higher frequency than quarterly scores.

# Matching 7-Day Windows to Target Scores

We associate each generated 7-day window with a corresponding target ESG score:
- For each window, we search for the nearest predicted score within a 3-day tolerance window.
- If multiple candidates exist, the closest one is selected.

This step ensures that each input sequence is correctly paired with an output target for supervised training.

In [ ]:
df_preds = df_preds.copy()
df_preds["date"] = pd.to_datetime(df_preds["date"])

windows_df = pd.DataFrame({
    "index": range(len(X_7d)),
    "ticker": tickers_7d,
    "category": categories_7d,
    "target_date": target_dates_7d
})

matched_indices = []
scores = []

for i, row in tqdm(windows_df.iterrows(), total=len(windows_df), desc="🔍 Buscando targets con tolerancia"):
    tkr = row["ticker"]
    cat = row["category"]
    date = row["target_date"]

    df_sub = df_preds[(df_preds["ticker"] == tkr) & (df_preds["category"] == cat)].copy()
    df_sub["date_diff"] = (df_sub["date"] - date).abs()
    df_sub = df_sub[df_sub["date_diff"] <= pd.Timedelta(days=3)]

    if not df_sub.empty:
        best_row = df_sub.sort_values("date_diff").iloc[0]
        matched_indices.append(i)
        scores.append(best_row["predicted_score"])

X_lstm = X_7d[matched_indices]
y_lstm = np.array(scores)

🔍 Buscando targets con tolerancia:   0%|          | 0/69295 [00:00<?, ?it/s]

In [ ]:
class ESG_LSTM(nn.Module):
    def __init__(self, input_dim=101, hidden_dim=128, num_layers=2, bidirectional=True):
        super(ESG_LSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_dim,
            hidden_dim,
            num_layers,
            batch_first=True,
            bidirectional=bidirectional
        )
        lstm_output_dim = hidden_dim * (2 if bidirectional else 1)

        self.fc = nn.Sequential(
            nn.ReLU(),
            nn.Linear(lstm_output_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        out, _ = self.lstm(x)
        last_out = out[:, -1, :]  # Último hidden state
        return self.fc(last_out).squeeze(1)

In [ ]:
X_temp, X_test, y_temp, y_test = train_test_split(
    X_lstm, y_lstm, test_size=0.15, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.1765, random_state=42
)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_ds = TensorDataset(X_train_tensor, y_train_tensor)
val_ds = TensorDataset(X_val_tensor, y_val_tensor)
test_ds = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 64

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ESG_LSTM(input_dim=X_lstm.shape[2]).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

epochs = 100
patience = 10
best_val_loss = float("inf")
epochs_without_improvement = 0

for epoch in range(epochs):
    model.train()
    train_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)
    train_loss /= len(train_loader.dataset)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb)
            loss = criterion(preds, yb)
            val_loss += loss.item() * xb.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict()
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

model.load_state_dict(best_model_state)
print("Mejor modelo cargado tras entrenamiento.")

Epoch 1/100 | Train Loss: 362.8534 | Val Loss: 55.9026
Epoch 2/100 | Train Loss: 51.3030 | Val Loss: 25.6027
Epoch 3/100 | Train Loss: 13.4062 | Val Loss: 9.1150
Epoch 4/100 | Train Loss: 7.2796 | Val Loss: 5.9908
Epoch 5/100 | Train Loss: 5.2398 | Val Loss: 5.3076
Epoch 6/100 | Train Loss: 4.2508 | Val Loss: 4.5575
Epoch 7/100 | Train Loss: 3.4137 | Val Loss: 3.7268
Epoch 8/100 | Train Loss: 3.0070 | Val Loss: 4.1328
Epoch 9/100 | Train Loss: 2.7202 | Val Loss: 3.1213
Epoch 10/100 | Train Loss: 2.3246 | Val Loss: 2.5431
Epoch 11/100 | Train Loss: 2.3614 | Val Loss: 2.7791
Epoch 12/100 | Train Loss: 1.9391 | Val Loss: 2.4256
Epoch 13/100 | Train Loss: 1.8389 | Val Loss: 2.2798
Epoch 14/100 | Train Loss: 1.6872 | Val Loss: 2.2144
Epoch 15/100 | Train Loss: 1.8067 | Val Loss: 2.2564
Epoch 16/100 | Train Loss: 1.4657 | Val Loss: 2.0565
Epoch 17/100 | Train Loss: 1.2726 | Val Loss: 1.8951
Epoch 18/100 | Train Loss: 1.2215 | Val Loss: 1.8422
Epoch 19/100 | Train Loss: 1.2081 | Val Loss: 1.7

In [ ]:
model.eval()

all_preds, all_targets = [], []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X)

        all_preds.extend(outputs.cpu().numpy())
        all_targets.extend(batch_y.cpu().numpy())

y_pred_real = np.array(all_preds)
y_true_real = np.array(all_targets)

# Métricas
mse = mean_squared_error(y_true_real, y_pred_real)
r2 = r2_score(y_true_real, y_pred_real)

print(f"Test MSE: {mse:.4f}")
print(f"Test R²: {r2:.4f}")

Test MSE: 0.4569
Test R²: 0.9917


In [ ]:
X_pred, _, tickers_pred, categories_pred, dates_pred = generate_windows_with_padding(df_embeddings, window_size=7)

Generando ventanas con padding:   0%|          | 0/90 [00:00<?, ?it/s]

In [ ]:
model.eval()
batch_size = 128
predictions = []

with torch.no_grad():
    for i in range(0, len(X_pred), batch_size):
        batch = torch.tensor(X_pred[i:i+batch_size], dtype=torch.float32).to(device)
        pred = model(batch).cpu().numpy().flatten()
        predictions.extend(pred)

In [ ]:
df_preds_lstm_7d = pd.DataFrame({
    "ticker": tickers_pred,
    "date": dates_pred,
    "category": categories_pred,
    "predicted_score": predictions
})

In [ ]:
df_preds["date"] = pd.to_datetime(df_preds["date"])
df_preds_lstm_7d["date"] = pd.to_datetime(df_preds_lstm_7d["date"])

df_preds["key"] = df_preds["ticker"] + "_" + df_preds["category"] + "_" + df_preds["date"].astype(str)
df_preds_lstm_7d["key"] = df_preds_lstm_7d["ticker"] + "_" + df_preds_lstm_7d["category"] + "_" + df_preds_lstm_7d["date"].astype(str)

nuevas_preds = df_preds_lstm_7d[~df_preds_lstm_7d["key"].isin(df_preds["key"])].drop(columns="key")
df_preds = df_preds.drop(columns="key")
df_preds_completo = pd.concat([df_preds, nuevas_preds], ignore_index=True).sort_values(["ticker", "category", "date"])

In [ ]:
conteo = df_preds_completo.groupby(['ticker', 'category']).size().unstack(fill_value=0)
display(conteo)

category,Environmental,Governance,Social
ticker,,,
AAPL,1500,1687,1614
ABBV,27,201,420
AMZN,1631,1579,1681
AXP,177,339,471
CMCSA,180,456,575
COP,970,480,147
CVS,150,458,1068
CVX,1350,894,339
DIS,520,676,1286


In [ ]:
umbral_dias = 100
conteo_validas = (conteo >= umbral_dias).astype(int)
empresas_bien_cubiertas = conteo_validas.sum(axis=1)
tickers_validos = empresas_bien_cubiertas[empresas_bien_cubiertas >= 2].index.tolist()

In [ ]:
tickers_validos

['AAPL',
 'ABBV',
 'AMZN',
 'AXP',
 'CMCSA',
 'COP',
 'CVS',
 'CVX',
 'DIS',
 'GM',
 'GOOGL',
 'GS',
 'HD',
 'JNJ',
 'JPM',
 'MCD',
 'MPC',
 'MS',
 'MSFT',
 'NFLX',
 'NKE',
 'NVDA',
 'PFE',
 'SBUX',
 'TSLA',
 'UNH',
 'VLO',
 'VZ',
 'WFC',
 'XOM']

In [ ]:
df_filtrado = df_preds_completo[df_preds_completo["ticker"].isin(tickers_validos)]
conteo = df_filtrado.groupby(['ticker', 'category']).size().unstack(fill_value=0)
display(conteo)

category,Environmental,Governance,Social
ticker,,,
AAPL,1500,1687,1614
ABBV,27,201,420
AMZN,1631,1579,1681
AXP,177,339,471
CMCSA,180,456,575
COP,970,480,147
CVS,150,458,1068
CVX,1350,894,339
DIS,520,676,1286


In [ ]:
df_filtrado["date"] = pd.to_datetime(df_filtrado["date"])
ultima_fecha = df_filtrado["date"].max()

In [ ]:
df_preds_completo["date"] = pd.to_datetime(df_preds_completo["date"])
fecha_inicio = pd.to_datetime("2020-03-23")
fecha_fin = pd.to_datetime("2025-03-23")

df_preds_filtrado = df_preds_completo[
    (df_preds_completo["ticker"].isin(tickers_validos)) &
    (df_preds_completo["date"] >= fecha_inicio) &
    (df_preds_completo["date"] <= fecha_fin)
].copy()

In [ ]:
def fill_esg_scores_filtered(df_preds, df_scores, tickers_validos, start_date, end_date):
    results = []
    categories = ["Environmental", "Social", "Governance"]
    score_map = {
        "Environmental": "environmentalScore",
        "Social": "socialScore",
        "Governance": "governanceScore"
    }

    for ticker in tickers_validos:
        for category in categories:
            score_col = score_map[category]

            df_p = df_preds[(df_preds["ticker"] == ticker) & (df_preds["category"] == category)].copy()
            df_s = df_scores[df_scores["symbol"] == ticker].copy()

            df_p["date"] = pd.to_datetime(df_p["date"])
            df_s["date"] = pd.to_datetime(df_s["date"])

            if score_col not in df_s.columns:
                continue

            score_series = df_s.groupby("date")[score_col].last().sort_index()

            if df_p.empty and score_series.empty:
                continue

            all_dates = pd.date_range(start=start_date, end=end_date, freq="D")
            df_base = pd.DataFrame({
                "date": all_dates,
                "ticker": ticker,
                "category": category
            })

            df_merge = pd.merge(df_base, df_p[["date", "predicted_score"]], on="date", how="left")

            has_predictions = df_merge["predicted_score"].notna().any()
            first_pred_date = (
                df_merge["date"][df_merge["predicted_score"].notna()].min()
                if has_predictions else end_date + pd.Timedelta(days=1)
            )

            final_scores = []
            from_official_flags = []
            last_pred = None

            for date in df_merge["date"]:
                if date < first_pred_date:
                    if not score_series.empty:
                        value = score_series.reindex([date], method="ffill").values[0]
                        from_official = True
                    else:
                        value = None
                        from_official = True
                else:
                    pred = df_merge.loc[df_merge["date"] == date, "predicted_score"].values[0]
                    if not pd.isna(pred):
                        last_pred = pred
                    value = last_pred
                    from_official = False if last_pred is not None else True

                final_scores.append(value)
                from_official_flags.append(from_official)

            df_merge["predicted_score_final"] = final_scores
            df_merge["from_official"] = from_official_flags
            results.append(df_merge)

    return pd.concat(results, ignore_index=True)

In [ ]:
esg_scores = pd.read_csv('/content/drive/MyDrive/MIAX/TFM/AAA RED NEURONAL/ESG_SCORES.csv')

In [ ]:
df_esg_final_5y = fill_esg_scores_filtered(
    df_preds=df_preds_filtrado,
    df_scores=esg_scores,
    tickers_validos=tickers_validos,
    start_date=fecha_inicio,
    end_date=fecha_fin
)

In [ ]:
df_esg_final_5y[df_esg_final_5y['ticker'] == 'CVS']

,date,ticker,category,predicted_score,predicted_score_final,from_official
32886,2020-03-23,CVS,Environmental,NaN,NaN,True
32887,2020-03-24,CVS,Environmental,NaN,NaN,True
32888,2020-03-25,CVS,Environmental,NaN,NaN,True
32889,2020-03-26,CVS,Environmental,NaN,NaN,True
32890,2020-03-27,CVS,Environmental,NaN,NaN,True
...,...,...,...,...,...,...
38362,2025-03-19,CVS,Governance,70.03889,70.03889,False
38363,2025-03-20,CVS,Governance,NaN,70.03889,False
38364,2025-03-21,CVS,Governance,NaN,70.03889,False
38365,2025-03-22,CVS,Governance,NaN,70.03889,False


In [ ]:
df_esg_final_5y.to_csv("/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/ESG_SCORES_COMPLETOS_2.csv", index=False)